In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LogisticRegression
import os
import joblib
import json

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense , Flatten, Dropout
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from kerastuner.tuners import RandomSearch

from sklearn.metrics import confusion_matrix, classification_report

# Set directory
base_dir = './sets'
# Load validation set and test set
X_val = np.load(f'{base_dir}/X_val.npy')
y_val = np.load(f'{base_dir}/y_val.npy')
X_test = np.load(f'{base_dir}/X_test.npy')
y_test = np.load(f'{base_dir}/y_test.npy')

# load best sets
over_sampled_best_X_val = np.load(f'{base_dir}/over_sampled_best_X_val.npy')
under_sampled_best_X_val = np.load(f'{base_dir}/under_sampled_best_X_val.npy')
over_sampled_best_X_test = np.load(f'{base_dir}/over_sampled_best_X_test.npy')
under_sampled_best_X_test = np.load(f'{base_dir}/under_sampled_best_X_test.npy')

# make output directory
output_dir = './deeplearning_out'
os.makedirs(output_dir, exist_ok=True)

n_features = X_val.shape[1]
n_best_over_features = over_sampled_best_X_val.shape[1]
n_best_under_features = under_sampled_best_X_val.shape[1]

2024-04-14 04:18:11.403581: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-14 04:18:12.314287: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/tmp/ipykernel_5554/2105926852.py:18: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


# Tuning Hyperparameters

In [4]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
                    activation='relu',
                    input_shape=(n_features,)))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=50,
    executions_per_trial=1,
    directory=output_dir,
    project_name='under_no_sel_shallow_nn'
)

early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)


2024-04-14 04:18:23.773704: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-14 04:18:23.926267: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-14 04:18:23.926519: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# UnderSampled Set

In [6]:
x_train = np.load(f'{base_dir}/under_sampled_X_train.npy')
y_train = np.load(f'{base_dir}/under_sampled_y_train.npy')

tuner.search(x_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping])

best_model = tuner.get_best_hyperparameters(num_trials=1)[0]

model = tuner.hypermodel.build(best_model)
history = model.fit(x_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping])

os.makedirs(f'{output_dir}/under_no_sel_deepL', exist_ok=True)

model.save(f'{output_dir}/under_no_sel_deepL/model.h5')
with open(f'{output_dir}/under_no_sel_deepL/history.json', 'w') as f:
    json.dump(history.history, f)
with open(f'{output_dir}/under_no_sel_deepL/best_model.json', 'w') as f:
    json.dump(best_model.values, f)

# Evaluate model
test_loss, test_acc = model.evaluate(X_test, y_test)

with open(f'{output_dir}/under_no_sel_deepL/test_results.json', 'w') as f:
    json.dump({'test_loss': test_loss, 'test_acc': test_acc}, f)

y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

conf_matrix = conf_matrix / conf_matrix.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='.2f', cmap='Blues')
plt.title('Confusion Matrix')
plt.show()

Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 115ms/step - accuracy: 0.0303 - loss: 3.2325 - val_accuracy: 0.0502 - val_loss: 2.2892
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.0649 - loss: 2.5573 - val_accuracy: 0.1180 - val_loss: 2.1726
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.2232 - loss: 1.8794 - val_accuracy: 0.2405 - val_loss: 2.0608
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.5168 - loss: 1.5102 - val_accuracy: 0.3813 - val_loss: 1.9579
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.6774 - loss: 1.2923 - val_accuracy: 0.5174 - val_loss: 1.8609
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 0.7467 - loss: 1.1534 - val_accuracy: 0.6376 - val_loss: 1.7683
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.7831 - loss: 1.1030 - val_accuracy: 0.7319 - val_loss: 1.6806
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - accuracy: 0.8296 - loss: 1.0450 - val_accuracy: 0

KeyboardInterrupt: 

# OverSampled Set

In [5]:
x_train = np.load(f'{base_dir}/over_sampled_X_train.npy')
y_train = np.load(f'{base_dir}/over_sampled_y_train.npy')

tuner.search(x_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping])

best_model = tuner.get_best_hyperparameters(num_trials=1)[0]

model = tuner.hypermodel.build(best_model)
history = model.fit(x_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stopping])

os.makedirs(f'{output_dir}/overSampled_no_sel_deepL', exist_ok=True)

model.save(f'{output_dir}/overSampled_no_sel_deepL/model.h5')
with open(f'{output_dir}/overSampled_no_sel_deepL/history.json', 'w') as f:
    json.dump(history.history, f)
with open(f'{output_dir}/overSampled_no_sel_deepL/best_model.json', 'w') as f:
    json.dump(best_model.values, f)

# Evaluate model
test_loss, test_acc = model.evaluate(X_test, y_test)

with open(f'{output_dir}/overSampled_no_sel_deepL/test_results.json', 'w') as f:
    json.dump({'test_loss': test_loss, 'test_acc': test_acc}, f)

y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

conf_matrix = conf_matrix / conf_matrix.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='.2f', cmap='Blues')
plt.title('Confusion Matrix')
plt.show()


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
352               |352               |units
0.0001            |0.0001            |learning_rate

Epoch 1/100


I0000 00:00:1713064708.580547    5668 service.cc:145] XLA service 0x732604006fb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1713064708.580583    5668 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-04-14 04:18:28.614479: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-14 04:18:28.692600: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 211/8086 ━━━━━━━━━━━━━━━━━━━━ 5s 720us/step - accuracy: 0.7957 - loss: 1.1947

I0000 00:00:1713064709.996052    5668 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


8086/8086 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9551 - loss: 0.2364 - val_accuracy: 0.9948 - val_loss: 0.0203
Epoch 2/100
8086/8086 ━━━━━━━━━━━━━━━━━━━━ 7s 856us/step - accuracy: 0.9975 - loss: 0.0140 - val_accuracy: 0.9972 - val_loss: 0.0099
Epoch 3/100
6027/8086 ━━━━━━━━━━━━━━━━━━━━ 1s 744us/step - accuracy: 0.9987 - loss: 0.0067

KeyboardInterrupt: 